Dans ce TP, nous considérons des trajets en vélo partagé (similaire au Vélib) en Californie. Deux jeux de données sont fournis : l'un qui contient les stations de vélo, l'autre, les trajets à vélo. Les déplacements à vélo se font d'une station à l'autre.

Charger le fichier stationData.csv dans un DataFrame `station_df` et le fichier tripData.csv dans un DataFrame `trip_df`. Pour chaque DataFrame, il vous faudra demander une inférence des schémas et indiquer que la première ligne est un en-tête.

In [ ]:
from pathlib import Path
import pandas as pd
import sqlite3

# Chemins vers les fichiers de données
data_dir = Path("../../data")
station_file = data_dir / "stationData.csv"
trip_file = data_dir / "tripData.csv"

# Chargement des fichiers
station_df = pd.read_csv(station_file)
trip_df = pd.read_csv(trip_file)

354152

Afficher les schémas/informations des deux DataFrames.

In [2]:
print("Schema de station_df:")
print(station_df.info())
print("\nSchema de trip_df:")
print(trip_df.info())

Schema de station_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   station_id    70 non-null     int64  
 1   name          70 non-null     object 
 2   lat           70 non-null     float64
 3   long          70 non-null     float64
 4   dockcount     70 non-null     int64  
 5   landmark      70 non-null     object 
 6   installation  70 non-null     object 
dtypes: float64(2), int64(2), object(3)
memory usage: 4.0+ KB
None

Schema de trip_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354152 entries, 0 to 354151
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   TripID          354152 non-null  int64 
 1   Duration        354152 non-null  int64 
 2   StartDate       354152 non-null  object
 3   StartStation    354152 non-null  object
 4   StartTerminal   

Créer une base SQLite ainsi qu'une vue SQL pour chaque DataFrame.

In [3]:
# Création de la base SQLite pour permettre les requêtes SQL
conn = sqlite3.connect(":memory:")
station_df.to_sql("station", conn, index=False, if_exists="replace")
trip_df.to_sql("trip", conn, index=False, if_exists="replace")

354152

Trouver deux façons de calculer le nombre de trajets, l'une en appelant une méthode sur `trip_df` directement, l'autre en rédigeant une requête SQL sur la vue correspondante.

In [4]:
# Méthode Pandas
print("Nombre de trajets (Pandas):", len(trip_df))

# Méthode SQL
query = "SELECT COUNT(*) AS trip_count FROM trip"
trip_count = pd.read_sql(query, conn)
print("Nombre de trajets (SQL):", trip_count.iloc[0]["trip_count"])

Nombre de trajets (Pandas): 354152
Nombre de trajets (SQL): 354152


Écrire une requête permettant de compter le nombre de trajets qui démarrent et se terminent à la même station.

In [5]:
# Méthode Pandas
same_station_count = len(trip_df[trip_df["StartStation"] == trip_df["EndStation"]])
print("Nombre de trajets démarrant et terminant à la même station (Pandas):", same_station_count)

# Méthode SQL
query = """
SELECT COUNT(*) AS same_station_trips
FROM trip
WHERE StartStation = EndStation
"""
same_station_trips = pd.read_sql(query, conn)
print("Nombre de trajets démarrant et terminant à la même station (SQL):", same_station_trips.iloc[0]["same_station_trips"])

Nombre de trajets démarrant et terminant à la même station (Pandas): 10276
Nombre de trajets démarrant et terminant à la même station (SQL): 10276


Obtenir l’id des stations associées à ces trajets. Écrire une requête renvoyant la liste des terminaux concernés ainsi que le nombre de trajets pour chacun de ces terminaux. Trier le résultat par ordre décroissant de nombre de trajets.

In [6]:
# Méthode Pandas
terminal_trips = (
    trip_df[trip_df["StartStation"] == trip_df["EndStation"]]
    .groupby("StartTerminal")
    .size()
    .reset_index(name="trip_count")
    .sort_values(by="trip_count", ascending=False)
)
print(terminal_trips)

# Méthode SQL
query = """
SELECT StartTerminal AS terminal, COUNT(*) AS trip_count
FROM trip
WHERE StartStation = EndStation
GROUP BY StartTerminal
ORDER BY trip_count DESC
"""
terminal_trips_sql = pd.read_sql(query, conn)
print(terminal_trips_sql)

    StartTerminal  trip_count
48             60         850
40             50         708
28             35         348
64             76         320
62             74         307
..            ...         ...
66             80          26
17             24          23
68             83          21
14             21          12
19             26           7

[70 rows x 2 columns]
    terminal  trip_count
0         60         850
1         50         708
2         35         348
3         76         320
4         74         307
..       ...         ...
65        80          26
66        24          23
67        83          21
68        21          12
69        26           7

[70 rows x 2 columns]


Compléter le résultat en indiquant le nombre de docks (dockcount) des stations concernées.

In [7]:
# Méthode Pandas
terminal_docks = (
    trip_df[trip_df["StartStation"] == trip_df["EndStation"]]
    .merge(station_df, left_on="StartTerminal", right_on="station_id")
    .groupby(["StartTerminal", "dockcount"])
    .size()
    .reset_index(name="trip_count")
    .sort_values(by="trip_count", ascending=False)
)
print(terminal_docks)

# Méthode SQL
query = """
SELECT t.StartTerminal AS terminal, s.dockcount, COUNT(*) AS trip_count
FROM trip t
JOIN station s ON t.StartTerminal = s.station_id
WHERE t.StartStation = t.EndStation
GROUP BY t.StartTerminal, s.dockcount
ORDER BY trip_count DESC
"""
terminal_docks_sql = pd.read_sql(query, conn)
print(terminal_docks_sql)

    StartTerminal  dockcount  trip_count
48             60         15         850
40             50         23         708
28             35         11         348
64             76         19         320
62             74         23         307
..            ...        ...         ...
66             80         15          26
17             24         15          23
68             83         15          21
14             21         15          12
19             26         15           7

[70 rows x 3 columns]
    terminal  dockcount  trip_count
0         60         15         850
1         50         23         708
2         35         11         348
3         76         19         320
4         74         23         307
..       ...        ...         ...
65        80         15          26
66        24         15          23
67        83         15          21
68        21         15          12
69        26         15           7

[70 rows x 3 columns]
